In [12]:
from setfit import SetFitModel, sample_dataset
import pandas as pd
from datasets import load_dataset
import numpy as np
import torch

### Data preperation and model training

In [2]:
model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5")

/home/xiyan/data/conda/envs/GPU/lib/python3.11/site-packages/transformers/utils/generic.py:319: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [45]:
dataset = pd.read_csv("scratch/3_SML_benchmark.csv")

In [46]:
dataset.tail()

,Unnamed: 0,index,filename,text_original,text_clean,date,year,month,outlet,country,south,RA
475,475,101008,newsarticle/Toronto Star/Neutral & Urgent 2006...,\n Business\nTime to invest in foreign stude...,Business Time to invest in foreign students Da...,2022-09-10,2022,9,Toronto Star,Canada,0,0
476,476,100986,newsarticle/Toronto Star/Neutral & Urgent 2006...,\n Politics\nMeet the Greens who might take ...,Politics Meet the Greens who might take a run ...,2022-07-04,2022,7,Toronto Star,Canada,0,0
477,477,2962,A cautionary flag for oil development;Study ca...,"\nMarch 1, 2020 Sunday\n\n\nCopyright 2020 Tor...","March 1, 2020 Sunday Copyright 2020 Toronto St...",2020-03-01,2020,3,Toronto Star,Canada,0,0
478,478,74551,The Toronto Star_2018-8-68.txt,""""";""x""\n""1"";"" 82\n\n\n ...",""""";""x"" ""1"";"" 82 The Toronto Star August 5, 201...",2018-08-01,2018,8,Toronto Star,Canada,0,1
479,479,72479,The Toronto Star_2011-7-21_39703.txt,""""";""x""\n""1"";""DATE: 21.7.2011\nHEADLINE: Securi...",""""";""x"" ""1"";""DATE: 21.7.2011 HEADLINE: Security...",2011-07-01,2011,7,Toronto Star,Canada,0,1


In [47]:
df = dataset[["text_clean", "RA"]]
df.rename(columns={'RA': 'label','text_clean':'text'}, inplace=True)

/sctmp/xiyan/ipykernel_2848872/675981800.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'RA': 'label','text_clean':'text'}, inplace=True)


In [63]:
#train = df.sample(frac = 0.7)
train = df.groupby('label').sample(n=150)
df2 = df.drop(train.index)
validation = df2.sample(frac = 0.5)
test = df2.drop(validation.index)

In [64]:
train.to_csv("scratch/train.csv")
validation.to_csv("scratch/validation.csv")
test.to_csv("scratch/test.csv")

In [65]:
print(len(train),len(validation),len(test))

300 90 90


In [66]:
dataset = load_dataset("csv", data_files={'train':'scratch/train.csv','validation':'scratch/validation.csv','test': 'scratch/test.csv'})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [67]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'label'],
        num_rows: 300
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'text', 'label'],
        num_rows: 90
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'label'],
        num_rows: 90
    })
})

In [68]:
#train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=50)
#train_dataset = dataset["train"]
eval_dataset = dataset["validation"]
test_dataset = dataset["test"]

In [57]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define custom function for computing metrics
def compute_metrics(y_pred, y_test):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [70]:
torch.cuda.empty_cache()

In [69]:
from setfit import TrainingArguments

args = TrainingArguments(
    batch_size=16,
    num_epochs=1,
)

In [73]:
from setfit import Trainer

for i in [5, 10, 25, 50, 75, 100, 125, 150]:
    print(i)
    train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=i)
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        metric=compute_metrics,
    )
    trainer.train()
    metrics = trainer.evaluate(eval_dataset)
    print(metrics)

5


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 60
  Batch size = 16
  Num epochs = 1
  Total optimization steps = 4


Step,Training Loss


***** Running evaluation *****


...

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 45300
  Batch size = 16
  Num epochs = 1
  Total optimization steps = 2832


Step,Training Loss


***** Running evaluation *****


{'accuracy': 0.8888888888888888, 'precision': 0.8995215311004785, 'recall': 0.8778280542986425, 'f1': 0.8842592592592593}


### Hyperparameter Optimization

In [74]:
from setfit import SetFitModel
from typing import Dict, Any

def model_init(params: Dict[str, Any]) -> SetFitModel:
    params = params or {}
    max_iter = params.get("max_iter", 100)
    solver = params.get("solver", "liblinear")
    params = {
        "head_params": {
            "max_iter": max_iter,
            "solver": solver,
        }
    }
    return SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5", **params)

In [106]:
from optuna import Trial
from typing import Dict, Union

def hp_space(trial: Trial) -> Dict[str, Union[float, int, str]]:
    return {
        "body_learning_rate": trial.suggest_float("body_learning_rate", 1e-5, 1e-3, log=True),
        "num_epochs": trial.suggest_int("num_epochs", 1, 3),
        "batch_size": trial.suggest_categorical("batch_size", [8, 16, 32, 64]),
        "seed": trial.suggest_int("seed", 1, 40),
        "max_iter": trial.suggest_int("max_iter", 50, 300),
        "solver": trial.suggest_categorical("solver", ["newton-cg", "lbfgs", "liblinear"]),
    }

In [107]:
train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=10)

trainer = Trainer(
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    model_init=model_init,
    metric=compute_metrics,
)
best_run = trainer.hyperparameter_search(direction="maximize", hp_space=hp_space, n_trials=10)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

[I 2024-03-18 17:24:13,653] A new study created in memory with name: no-name-79619705-441e-45a3-88c5-e21a85cd7b3c
Trial: {'body_learning_rate': 1.1065254207186703e-05, 'num_epochs': 2, 'batch_size': 8, 'solver': 'newton-cg'}
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num unique pairs = 220
  Batch size = 8
  Num epochs = 2
  Total optimization steps = 56


Step,Training Loss


***** Running evaluation *****
[I 2024-03-18 17:24:23,377] Trial 0 finished with value: 2.8595674513828735 and parameters: {'body_learning_rate': 1.1065254207186703e-05, 'num_epochs': 2, 'batch_size': 8, 'solver': 'newton-cg'}. Best is trial 0 with value: 2.8595674513828735.
Trial: {'body_learning_rate': 1.1216026167266768e-05, 'num_epochs': 2, 'batch_size': 64, 'solver': 'lbfgs'}
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num unique pairs = 220
  Batch size = 64
  Num epochs = 2
  Total optimization steps = 8


...

***** Running evaluation *****
[I 2024-03-18 17:25:30,370] Trial 7 finished with value: 3.2325796215100073 and parameters: {'body_learning_rate': 0.0003103369900521842, 'num_epochs': 2, 'batch_size': 32, 'solver': 'newton-cg'}. Best is trial 7 with value: 3.2325796215100073.
Trial: {'body_learning_rate': 0.00020750426513882773, 'num_epochs': 2, 'batch_size': 64, 'solver': 'lbfgs'}
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num unique pairs = 220
  Batch size = 64
  Num epochs = 2
  Total optimization steps = 8


Step,Training Loss


***** Running evaluation *****
[I 2024-03-18 17:25:39,006] Trial 8 finished with value: 2.733119195619926 and parameters: {'body_learning_rate': 0.00020750426513882773, 'num_epochs': 2, 'batch_size': 64, 'solver': 'lbfgs'}. Best is trial 7 with value: 3.2325796215100073.
Trial: {'body_learning_rate': 1.6459223544468137e-06, 'num_epochs': 3, 'batch_size': 8, 'solver': 'liblinear'}
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num unique pairs = 220
  Batch size = 8
  Num epochs = 3
  Total optimization steps = 84


Step,Training Loss


***** Running evaluation *****
[I 2024-03-18 17:25:52,916] Trial 9 finished with value: 2.9031098015585766 and parameters: {'body_learning_rate': 1.6459223544468137e-06, 'num_epochs': 3, 'batch_size': 8, 'solver': 'liblinear'}. Best is trial 7 with value: 3.2325796215100073.


In [108]:
print(best_run)

BestRun(run_id='7', objective=3.2325796215100073, hyperparameters={'body_learning_rate': 0.0003103369900521842, 'num_epochs': 2, 'batch_size': 32, 'solver': 'newton-cg'}, backend=<optuna.study.study.Study object at 0x15256eb161d0>)


In [109]:
trainer.apply_hyperparameters(best_run.hyperparameters, final_model=True)
trainer.train()

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num unique pairs = 220
  Batch size = 32
  Num epochs = 2
  Total optimization steps = 14


Step,Training Loss


In [110]:
trainer.evaluate(eval_dataset)

***** Running evaluation *****


{'accuracy': 0.8111111111111111,
 'precision': 0.8081983805668016,
 'recall': 0.806184012066365,
 'f1': 0.8070861177657295}

In [111]:
trainer.evaluate(test_dataset)

***** Running evaluation *****


{'accuracy': 0.7555555555555555,
 'precision': 0.7570509648688768,
 'recall': 0.7570509648688768,
 'f1': 0.7555555555555555}

### Making predictions

In [ ]:
preds = model.predict()